In [1]:
import torch 
import numpy as np
from Loss import Loss
from RON import RecurrentOccamNet
from RONTrainer import RONTrainer
from cocob import COCOBBackprop 
from bases import *
import pandas as pd

alpha = 0.1
NUM_STEPS = 10  

rewards = pd.read_csv('/Users/thomasnegrello/Desktop/autoneuro/data_to_share/classical/overshadowing/input/rewards.csv', header=None).to_numpy()
observations = pd.read_csv('/Users/thomasnegrello/Desktop/autoneuro/data_to_share/classical/overshadowing/input/states.csv', header=None).to_numpy()
rpes = pd.read_csv(f'/Users/thomasnegrello/Desktop/autoneuro/data_to_share/classical/overshadowing/output/RW_alpha={alpha}00/rpes.csv', header=None)  
observations = observations.copy()
observations = np.concatenate((np.zeros((1, observations.shape[1])), observations[:-1,:]), axis=0)

# input data
X = np.concatenate((rewards, observations), axis=1)[:NUM_STEPS,:]


# output data
y = rpes.to_numpy()[:NUM_STEPS,:]
y *= alpha  

In [2]:
input_dim = X.shape[1]
output_dim = y.shape[1]
hidden_dim = X.shape[1]

In [3]:
print(input_dim, output_dim)

51 1


In [4]:
r_model = RecurrentOccamNet(input_dim, hidden_dim, output_dim,
 [[on_sum(), on_sub(),on_mul(),on_div()], 
 [on_sum(), on_sub(),on_mul(),on_div()],
 [on_sum(), on_sub(),on_mul(),on_div()],
 [on_identity()]*(output_dim+hidden_dim)], 
 [0.01, 0.01, 0.01, 0.01], 
 skip_connections=False, is_none=False)

criterion = Loss(0.1)
optimizer = COCOBBackprop(r_model.parameters())
trainer = RONTrainer(r_model, criterion, optimizer)

trained_model = trainer.fit(X, y, 10, 3)

/Users/thomasnegrello/Desktop/autoneuro/occamnet_branches/brouillon/trial/RONTrainer.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  Y = torch.tensor([[y] * nb_sampled_paths for y in Y])
/Users/thomasnegrello/Desktop/autoneuro/occamnet_branches/brouillon/trial/Loss.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mse = torch.tensor((prediction-target)**2, requires_grad=False)


In [5]:
paths = trained_model.sample_paths(3)
paths

[tensor([[62, 61, 83, 33,  5,  5, 52,  7],
         [36, 60, 11, 54, 86, 24,  1, 75],
         [93,  0, 88,  6, 77, 78,  1,  8]]),
 tensor([[3, 3, 3, 3, 2, 3, 2, 0],
         [2, 1, 2, 3, 2, 1, 0, 1],
         [2, 3, 1, 1, 3, 2, 3, 1]]),
 tensor([[0, 0, 1, 3, 1, 1, 2, 1],
         [0, 3, 1, 3, 1, 0, 0, 2],
         [1, 3, 1, 3, 1, 2, 3, 1]]),
 tensor([[2, 2, 2, 0, 1, 1, 1, 3, 2, 2, 1, 1, 3, 1, 1, 0, 2, 1, 3, 0, 1, 1, 0, 1,
          0, 3, 3, 2, 2, 1, 0, 0, 3, 3, 2, 0, 0, 3, 1, 2, 2, 2, 0, 1, 2, 0, 2, 0,
          3, 1, 1, 1],
         [0, 3, 1, 0, 1, 0, 0, 3, 2, 2, 1, 2, 3, 0, 2, 2, 1, 3, 2, 2, 0, 3, 2, 3,
          2, 2, 1, 0, 2, 0, 3, 2, 1, 3, 0, 2, 0, 3, 2, 1, 3, 0, 2, 2, 2, 2, 3, 0,
          2, 0, 2, 0],
         [3, 1, 3, 2, 1, 2, 3, 1, 2, 1, 2, 1, 1, 3, 0, 3, 3, 2, 3, 0, 3, 2, 0, 1,
          0, 2, 1, 1, 1, 2, 2, 3, 3, 3, 2, 2, 3, 3, 3, 2, 0, 1, 3, 1, 3, 0, 1, 1,
          1, 3, 0, 0]])]

In [6]:
paths[-1].shape

torch.Size([3, 52])